<a href="https://colab.research.google.com/github/iisc-ts-group6/capstone/blob/main/SBERT_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers
!pip install datasets
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 4

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Load the dataset from the uploaded Excel file
file_path = '/content/master_sheet_use.xlsx'
df = pd.read_excel(file_path)

# Ensure all columns are of the correct type (string)
for col in ['Answer', 'Para1', 'Para2', 'D1', 'D2', 'D3']:
    df[col] = df[col].astype(str)

# Create pairs of texts and labels
pairs = []
labels = []

for index, row in df.iterrows():
    pairs.append((row['Answer'], row['Para1']))
    labels.append(1)
    pairs.append((row['Answer'], row['Para2']))
    labels.append(1)
    pairs.append((row['Answer'], row['D1']))
    labels.append(0)
    pairs.append((row['Answer'], row['D2']))
    labels.append(0)
    pairs.append((row['Answer'], row['D3']))
    labels.append(0)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
df.head()

,Question,Answer,Para1,Para2,D1,D2,D3
0,What are some examples of young animals you h...,Some examples of young animals being born that...,I have observed the birth of certain young ani...,Chicks and caterpillars are two types of young...,I have seen baby seahorses being born.,I once witnessed newborn baby butterflies emer...,I remember watching baby penguins hatch from e...
1,What is the importance of reproduction in ani...,The importance of reproduction in animals for ...,One reason why animal reproduction is essentia...,Animals reproduce to maintain the presence of ...,The importance of reproduction in animals for ...,Reproduction in animals is crucial because it ...,The primary purpose of reproduction in animals...
2,How does reproduction in animals differ from ...,Reproduction in animals typically involves int...,Animal reproduction commonly includes internal...,"In plants, reproduction usually starts with po...",Reproduction in animals involves photosynthesi...,"Animals reproduce by forming seeds and fruits,...",Animals have the ability to self-pollinate dur...
3,Can you recall the modes of reproduction in a...,The modes of reproduction in animals discussed...,This chapter covers the different ways animals...,The chapter explains the methods by which anim...,"Yes, the modes of reproduction in animals disc...",The modes of reproduction in animals discussed...,"Absolutely, the modes of reproduction in anima..."
4,Have you observed the birth process of chicks...,"Yes, the birth process of chicks involves hatc...","Chicks are born by hatching out of eggs, and s...",The way chicks and caterpillars are born is by...,"No, I haven't observed the birth process of ch...",I haven't witnessed the birth process of chick...,I'm not familiar with the birth process of chi...


In [4]:
labels[0]

1

In [5]:
# Split the dataset into training and testing sets (80:20)
train_pairs, test_pairs, train_labels, test_labels = train_test_split(pairs, labels, test_size=0.2, random_state=42)

In [6]:
train_labels[0]

1

In [7]:
# Convert pairs and labels to InputExample format

train_examples = [InputExample(texts=[pair[0], pair[1]], label=label) for pair, label in zip(train_pairs, train_labels)]
test_examples = [InputExample(texts=[pair[0], pair[1]], label=label) for pair, label in zip(test_pairs, test_labels)]


In [8]:
train_examples[1]

In [9]:
# Load pre-trained SBERT model
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Create a DataLoader for training
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=model)

**SET up GPU for training the model**

In [ ]:
import torch
# specify GPU
device = torch.device("cuda")

In [11]:
# Fine-tune the model
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          show_progress_bar=True)

Step,Training Loss


In [14]:
# Save the fine-tuned model
model_save_path = 'finetuned_sbert_model'
model.save(model_save_path)

**Predict using trained model**

In [22]:

from sentence_transformers import SentenceTransformer
import numpy as np

# Load the fine-tuned model
model = SentenceTransformer(model_save_path)

In [24]:
# Evaluate the model on the test set
test_sentences1 = [example.texts[0] for example in test_examples]
test_sentences2 = [example.texts[1] for example in test_examples]
test_labels = [example.label for example in test_examples]

In [25]:
# Compute embeddings for test set
embeddings1 = model.encode(test_sentences1)
embeddings2 = model.encode(test_sentences2)

In [26]:
# Compute cosine similarities
cosine_similarities = [np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
                       for emb1, emb2 in zip(embeddings1, embeddings2)]


In [27]:
# Threshold for determining equivalence (you may need to adjust this)
threshold = 0.5

# Make predictions
test_predictions = [1 if sim >= threshold else 0 for sim in cosine_similarities]

# Calculate accuracy
accuracy = np.mean(np.array(test_predictions) == np.array(test_labels))
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 95.28%


In [29]:
# Create a DataFrame to view the results
results_df = pd.DataFrame({
    'sentence_1': test_sentences1,
    'sentence_2': test_sentences2,
    'label': test_labels,
    'prediction': test_predictions,
    'cosine_similarity': cosine_similarities
})

In [35]:
results_df
results_df.to_excel('qa_results.xlsx', index=False)

In [28]:
test_predictions

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,


In [36]:
# Calculate additional metrics
from sklearn.metrics import classification_report, confusion_matrix

# Generate classification report
report = classification_report(test_labels, test_predictions, target_names=['not_equivalent', 'equivalent'])
print("\nClassification Report:\n", report)

# Generate confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)
print("\nConfusion Matrix:\n", conf_matrix)


Classification Report:
                 precision    recall  f1-score   support

not_equivalent       0.99      0.94      0.96       443
    equivalent       0.91      0.98      0.94       277

      accuracy                           0.95       720
     macro avg       0.95      0.96      0.95       720
  weighted avg       0.96      0.95      0.95       720


Confusion Matrix:
 [[415  28]
 [  6 271]]


In [16]:
# new statements
new_statements = [("Photosynthesis needs carbon dioxide and sunlight", "Photosynthesis needs carbon monoxide and sunlight")]
                    # (" statement 2", "Comparison statement 2")]

In [17]:
# Compute sentence embeddings
embeddings1 = model.encode([pair[0] for pair in new_statements])
embeddings2 = model.encode([pair[1] for pair in new_statements])


In [18]:
# Compute cosine similarity
cosine_similarities = [np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
                       for emb1, emb2 in zip(embeddings1, embeddings2)]


In [21]:
cosine_similarities

[0.782346]

In [19]:
# Threshold for determining equivalence (can be adjusted based on validation performance)
threshold = 0.5

In [20]:
# Make predictions
predictions = [1 if sim >= threshold else 0 for sim in cosine_similarities]
print(predictions)

[1]


In [56]:
# NOT WORKING!!!!!!!!!!!!!!

def cosine_similarity(embeddings):
  similarities = []
  for i in range(len(embeddings) - 1):
    similarities.append(np.dot(embeddings[i], embeddings[i+1]) /
                       (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[i+1])))
  return similarities

# Function to test the model on new inputs
def test_similarity(correct_answer, student_answer):
    # Clean the input text

    #correct_answer = clean_text(correct_answer)
    #student_answer = clean_text(student_answer)

    # Compute embeddings
    embeddings = model.encode([correct_answer, student_answer])

    # Compute similarity score between correct answer and student answer
    sim_score = cosine_similarity(embeddings[1], embeddings[2])

    return sim_score

In [104]:
# Function to compute cosine similarity
def cosine_similarity(embeddings1, embeddings2):
    return [np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
            for emb1, emb2 in zip(embeddings1, embeddings2)]

def test_similarity(correct_answer, student_answer):
  # ... (text cleaning steps)

  # Assuming model returns a list with exactly 2 embeddings
  embeddings = model.encode([correct_answer, student_answer])
  correct_embedding, student_embedding = embeddings

  # Compute similarity score
  sim_score = np.dot(correct_embedding, student_embedding) / (np.linalg.norm(correct_embedding) * np.linalg.norm(student_embedding))

  return sim_score

In [106]:
correct_answer = "The different types of crops include cereals, vegetables, and fruits."
student_answer = "The different types of crops include cereals and fruits"

# Predict the model on the inputs
similarity_score = test_similarity(correct_answer, student_answer)
similarity_score

0.57715183

In [60]:
correct_answer = "When dead plants and animals decompose in the soil, soil organisms break them down, releasing nutrients back into the soil. These nutrients are then absorbed by plants, contributing to the fertility of the soil"
student_answer = "When dead plants and animals decompose in the soil, soil organisms break them down and release nutrients back into the soil and human can then eat that soil "

# Predict the model on the inputs
similarity_score = test_similarity(correct_answer, student_answer)
similarity_score

0.68903595

In [61]:
correct_answer = "Proper management of crop production is important to ensure regular production and distribution of food for a large population. Agricultural practices, such as cultivating crops on a large scale, help in meeting the food demands of society"
student_answer = "Proper Management of crop production is important because from crop we make foods and foods give us energy "

# Predict the model on the inputs
similarity_score = test_similarity(correct_answer, student_answer)
similarity_score

0.16723058

In [107]:
correct_answer = "Fertilisers have helped farmers achieve better crop yields by providing essential nutrients like phosphorus and potassium to the soil, which in turn promotes the growth and development of crops such as wheat, paddy, and maize."
student_answer = "Fertilisers help farmers achieve better crop yields by providing specific nutrients like Phosphorus and Potassium to the soil, which leads to improved growth and production."

# Predict the model on the inputs
similarity_score = test_similarity(correct_answer, student_answer)
similarity_score

0.93658245

# Monitor and retrain the model

In [69]:
def compute_cosine_similarities(embeddings1, embeddings2):
    return [np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
            for emb1, emb2 in zip(embeddings1, embeddings2)]




In [98]:
def add_feedback_to_dataset(feedback, pairs, labels):
    """
    Add feedback to the existing dataset.

    Args:
    feedback (list of dict): Feedback data containing pairs of sentences and labels.
    pairs (list of tuple): Existing pairs of sentences.
    labels (list of int/float): Existing labels.

    Returns:
    tuple: Updated pairs and labels.
    """
    # Ensure pairs and labels are lists (if necessary)
    if not isinstance(pairs, list):
      pairs = list(pairs)
    if not isinstance(labels, list):
      labels = list(labels)

  # Check feedback structure
    for item in feedback:
      if not isinstance(item, dict):
        raise ValueError("Feedback elements must be dictionaries")

    for item in feedback:
      pairs.append((item["sentence_1"], item["sentence_2"]))
      labels.append(item['label'])
    return pairs, labels


In [ ]:
#sentence_1 = item['sentence_1']
    #sentence_2 = item['sentence_2']
    for item in feedback:
        #pairs.append((sentence_1, sentence_2))
        pairs.append((item['sentence_1'], item['sentence_2']))
        labels.append(item['label'])
    return pairs, labels

In [101]:
# Sample feedback for demonstration
feedback = [
    {
        "sentence_1": "The different types of crops include cereals, vegetables, and fruits.",
        "sentence_2": "The different types of crops include cereals and fruits.",
        "label": 0.5  # partially correct
    },
    {
        "sentence_1": "When dead plants and animals decompose in the soil, soil organisms break them down, releasing nutrients back into the soil. These nutrients are then absorbed by plants, contributing to the fertility of the soil",
        "sentence_2": "When dead plants and animals decompose in the soil, soil organisms break them down and release nutrients back into the soil and human can then eat that soil",
        "label": 0.5   # partially correct
    },
    # Add more feedback here
]

In [100]:
# Update training data with feedback
train_pairs, train_labels = add_feedback_to_dataset(feedback, train_pairs, train_labels)


In [102]:
# Retrain the model with the updated dataset
train_examples = [InputExample(texts=[pair[0], pair[1]], label=label) for pair, label in zip(train_pairs, train_labels)]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=model)



In [103]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          show_progress_bar=True)

# Save the retrained model
model.save(model_save_path)

Step,Training Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [105]:
correct_answer = "Fertilisers have helped farmers achieve better crop yields by providing essential nutrients like phosphorus and potassium to the soil, which in turn promotes the growth and development of crops such as wheat, paddy, and maize."
student_answer = "Fertilisers help farmers achieve better crop yields by providing specific nutrients like Phosphorus and Potassium to the soil, which leads to improved growth and production."

# Predict the model on the inputs
similarity_score = test_similarity(correct_answer, student_answer)
similarity_score

0.93658245

In [108]:
correct_answer = "The different types of crops include cereals, vegetables, and fruits."
student_answer = "The different types of crops include cereals and fruits"

# Predict the model on the inputs
similarity_score = test_similarity(correct_answer, student_answer)
similarity_score

0.57715183